In [5]:
import sys
IN_COLAB = 'google.colab' in sys.modules

In [6]:
import json
from typing import NamedTuple
import base64
import requests
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set()

In [7]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [8]:
def get_config_collab(url):
    return json.loads(requests.get(url).text)

def get_config_local(filename):
    with open(filename) as f:
        raw = json.load(f)
    return raw

def load_config(filename):
    if IN_COLAB:
        return get_config_collab(f'https://raw.githubusercontent.com/Quick-One/Learning_AI-ML/main/MNIST_NN/{filename}')
    return get_config_local(filename)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using '{device}' device")

<hr>

### Loading the MNIST Dataset


In [ ]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)


In [ ]:
figure = plt.figure(figsize=(12, 1))
cols, rows = 9, 1
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(f'Number {label}')
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

<hr>

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, linear_relu_stack : nn.Sequential):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = linear_relu_stack

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # if batch % 250 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn, m):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    m.error_avg.append(test_loss)


<hr>

In [ ]:
class Model(NamedTuple):
    name: str
    learning_rate: float
    batch_size: int
    epochs: int
    hidden_layers: list
    error_avg : list

    def constructor(self):
        layer_size = self.hidden_layers.copy()
        layer_size.insert(0, 28 * 28)
        layer_size.append(10)

        layers = []
        for i in range(len(layer_size)-2):
            layers.append(nn.Linear(layer_size[i], layer_size[i+1]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(layer_size[-2], layer_size[-1]))

        linear_stack = nn.Sequential(*layers)
        return NeuralNetwork(linear_stack)


def load_models(config_dict: str) -> list:
    l = []
    for model_name, param in config_dict.items():
        error_arr = []
        m = Model(
            name=model_name,
            learning_rate=param['learning_rate'],
            batch_size=param['batch_size'],
            epochs=param['epochs'],
            hidden_layers=param['hidden_layers'],
            error_avg=error_arr
        )
        l.append(m)
    return l


<hr>

In [ ]:
def train_models(config_dict):
    models = []
    for m in load_models(config_dict):
        models.append(m)
    for m in models:
        print(f'Training {m.name}')
        model = m.constructor().to(device)
        train_dataloader = DataLoader(training_data, batch_size=m.batch_size)
        test_dataloader = DataLoader(test_data, batch_size=m.batch_size)
        loss_fn = nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=m.learning_rate)

        for t in range(m.epochs):
            print(f"Epoch {t+1}\n-------------------------------")
            train_loop(train_dataloader, model, loss_fn, optimizer)
            test_loop(test_dataloader, model, loss_fn, m)
    return models

In [ ]:
def visualise_model_perf(models : list, title = 'TITLE'):
    fig, ax = plt.subplots()
    for m in models:
        epochs = m.epochs
        ax.plot(range(1, epochs+1), m.error_avg, label = m.name)
    ax.set_title(title)
    ax.legend()
    return fig, ax

<hr>

In [ ]:
models = train_models(load_config('layers.json'))

In [ ]:
visualise_model_perf(models, title='Avg error vs Epoch')